In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
!tar xf spark-3.3.2-bin-hadoop3.tgz
!pip install -q findspark

In [4]:
#All imports
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

In [5]:
import findspark
findspark.init()
from google.colab import files
from pyspark import SparkContext, SparkConf,SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, lit
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 

In [6]:
# create SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

In [7]:
spark.version

'3.3.2'

In [8]:
#And SparkContext
#sc = spark._sc
sc = spark.sparkContext

In [9]:
sc.version

'3.3.2'

In [ ]:
#sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

In [ ]:
sqlContext = SQLContext(sc)

# <center>Getting Started with PySpark</center>

## Introduction

Apache Spark is a fast and powerful framework that provides an API to perform massive distributed processing over resilient sets of data. The main abstraction Spark provides is a resilient distributed dataset (RDD), which is the fundamental and backbone data type of this engine. Spark SQL is Apache Spark's module for working with structured data and MLlib is Apache Spark's scalable machine learning library. Apache Spark is written in Scala programming language. To support Python with Spark, Apache Spark community released a tool, PySpark. PySpark has similar computation speed and power as Scala. PySpark is a parallel and distributed engine for running big data applications. Using PySpark, you can work with RDDs in Python programming language. 

This tutorial discusses Big Data via PySpark, a Python package for spark programming. Spark's high level libraries such as SparkSQL, Mllib will be used to interact with two different datasets.

## What is SparkContext?

Spark comes with interactive python shell in which PySpark is already installed in it. PySpark automatically creates a SparkContext for you in the PySpark Shell. SparkContext is an entry point into the world of Spark. An entry point is a way of connecting to Spark cluster. We can use SparkContext using **sc** variable. In the following examples, we retrieve SparkContext version and Python version of SparkContext.

In [ ]:
# to retrieve SparkContext version
sc.version

'2.4.8'

In [ ]:
# to retriece Python version of SparkContext
sc.pythonVer

'3.7'

## Import Packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("seaborn")
from math import sqrt
from pyspark.mllib.clustering import KMeans

## Using map and filter methods with Lambda function in Python

Lambda functions are anonymous functions in Python. Anonymous functions do not bind to any name in runtime and it returns the functions without any name. They are usually used with map and filter methods. Lambda functions create functions to be called later. In the following example, we use lambda function with map and flter methods.

In [ ]:
my_list = [1,2,3,4,5]
squared_my_list = list(map(lambda x: x*x, my_list))
squared_my_list

[1, 4, 9, 16, 25]

In [ ]:
filtered_my_list = list(filter(lambda x: (x%2 != 0), my_list))
filtered_my_list

[1, 3, 5]

## Creating RDD from Object

RDDs are data stacks distributed throughout cluster of computers. Each stack is calculated on different computers in the cluster. RDDs are the most basic data structure of Spark. We can create RDD by giving existing object like Python list to SparkContext's parallelize method. In the following example, we create a list with numbers, then we create a RDD from this list.

In [ ]:
numbers = list(range(0,10))
print(numbers)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
# load the numbers into PySpark
numbersRDD = sc.parallelize(numbers)

In [ ]:
print(type(numbersRDD))
print(type(numbers))

<class 'pyspark.rdd.RDD'>
<class 'list'>


## Transformations and Actions on RDD

Transformations and actions are two type of operations in Spark. Transformations create new RDDs. Actions performs computation on the RDDs. Map, filter, flatmap and union are basic RDD transformations. Collect, take, first and count are basic RDD actions. In the following example, we create rdd named numRDD from list and then using map transformation we create a new rdd named cubeRDD from numRDD. Finally, we use collect action to return a list that contains all of the elements in this RDD.

In [ ]:
#Create RDD, map() transformations
numRDD = sc.parallelize(numbers)
cubeRDD = numRDD.map(lambda x: x**3)
numbers_all = cubeRDD.collect()

[print(num) for num in numbers_all];

0
1
8
27
64
125
216
343
512
729


In [ ]:
#take shows the first n values
numRDD.take(10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
#reduce reduces de values using a conmutative and associative binary operator
from operator import add
numRDD.reduce(add)

45

In [ ]:
numRDD.stats()

(count: 10, mean: 4.5, stdev: 2.8722813232690143, max: 9.0, min: 0.0)

In [ ]:
numRDD.count()

10

## Transformations and Actions on pair RDD

Pair RDD is a special type of RDD to work with datasets with key/value pairs. All regular transformations work on pair RDD. In the following example, we create pair RDD with 4 tuple with two numbers. In each tuple, the first number is key and the second number is value. Then, we apply reduceByKey transformation to pair RDD. ReduceByKey tranformation combine values with the same key. Therefore, this transformation adds the values of tuples with the same key.

In [ ]:
# create pairRdd with key and value pairs
pairRdd = sc.parallelize([(1,2), (3,4), (3,6), (3,7), (4,5)])

[print(num) for num in pairRdd.collect()];

print("\n")

pairRdd_Reduced = pairRdd.reduceByKey(lambda x, y: x+y)

[print(num) for num in pairRdd_Reduced.collect()];

(1, 2)
(3, 4)
(3, 6)
(3, 7)
(4, 5)


(4, 5)
(1, 2)
(3, 17)


In [ ]:
pairRdd.take(6)

[(1, 2), (3, 4), (3, 6), (3, 7), (4, 5)]

We can sort keys of tuples using sortByKey transformation like in the following example.

In [ ]:
pairRdd_Reduced_Sort = pairRdd_Reduced.sortByKey(ascending=True)

[print(num) for num in pairRdd_Reduced_Sort.collect()];

(1, 2)
(3, 17)
(4, 5)


In [ ]:
pairRdd.take(3)

[(1, 2), (3, 4), (3, 6)]

We can count number of tuples with the same key. In the following example, we see (3,2) because there are two tuple with key 3 in pairRdd.

In [ ]:
total = pairRdd.countByKey()

In [ ]:
total.items()

dict_items([(1, 1), (3, 3), (4, 1)])

In [ ]:
[print("key", k, "has", v, "counts") for k,v in total.items()];

key 1 has 1 counts
key 3 has 3 counts
key 4 has 1 counts


## What is SparkSession?

SparkContext is the main entry point for creating RDDs while SparkSession provides a single point of entry to interact with Spark Dataframes. SparkSession is used to create DataFrame, register DataFrames, execute SQL queries. We can access SparkSession in PySpark using spark variable.
In the following examples, we retrieve SparkSession version and other informations about it.

In [ ]:
spark

## Creating PySpark DataFrame from RDD

Spark SQL which is a Spark module for structured data processing provides a programming abstraction called DataFrames and can also act as a distributed SQL query engine. In the following example, we create rdd from list then we create PySpark dataframe using SparkSession's createDataFrame method. When we look at the type of dataframe, we can see pyspark.sql.dataframe as an output. Furthermore, we can use show method to print out the dataframe.

In [ ]:
# Create a PySpark Dataframe from rdd
sample_list = [("Mona", 23), ("Lisa", 29), ('Leonardo',37), ('Piero',41)]
rdd = sc.parallelize(sample_list)
df_names = spark.createDataFrame(rdd, schema=['Name', 'Age'])
type(df_names)

pyspark.sql.dataframe.DataFrame

In [ ]:
df_names.show()

+--------+---+
|    Name|Age|
+--------+---+
|    Mona| 23|
|    Lisa| 29|
|Leonardo| 37|
|   Piero| 41|
+--------+---+



In [ ]:
columns=["New Name","New Age"]
SDF=rdd.toDF(columns)
SDF.show()
type(SDF)

+--------+-------+
|New Name|New Age|
+--------+-------+
|    Mona|     23|
|    Lisa|     29|
|Leonardo|     37|
|   Piero|     41|
+--------+-------+



pyspark.sql.dataframe.DataFrame

In [ ]:
df_names.rdd.take(5)

[Row(Name='Mona', Age=23),
 Row(Name='Lisa', Age=29),
 Row(Name='Leonardo', Age=37),
 Row(Name='Piero', Age=41)]

In [ ]:
df_names.rdd.getNumPartitions()

2

## Add Datasets

You can delete df_data_1.take(5) part and then copy cos.url('file_name', 'bucket_name') above it then assign cos.url('file_name', 'bucket_name') to path_people variable and comment out this variable. cos.url('file_name', 'bucket_name') is path to your file you can access the file by using this path.

In [ ]:
#Mount Google drive, read local file. The file was previously uploaded to Google Drive
from google.colab import drive
drive.mount("/content/drive")
%ls

Mounted at /content/drive
drive/  sample_data/  spark-2.4.8-bin-hadoop2.7/  spark-2.4.8-bin-hadoop2.7.tgz


In [ ]:
path_people="/content/drive/MyDrive/Colab Notebooks/The Learning Gate/people.csv"

In [ ]:
df_data_1 = sqlContext.read.csv(path_people,header=True,inferSchema=True)
#df = spark.read.csv(path_csv,header=True,inferSchema=True).coalesce(2)
#df=spark.read.format("csv").option("header","true").option("inferSchema","true").option("minPartitions",2).load(path_csv)
df_data_1.show(10)

+---+---------+----------------+------+-------------+
|_c0|person_id|            name|   sex|date of birth|
+---+---------+----------------+------+-------------+
|  0|      100|  Penelope Lewis|female|   1990-08-31|
|  1|      101|   David Anthony|  male|   1971-10-14|
|  2|      102|       Ida Shipp|female|   1962-05-24|
|  3|      103|    Joanna Moore|female|   2017-03-10|
|  4|      104|  Lisandra Ortiz|female|   2020-08-05|
|  5|      105|   David Simmons|  male|   1999-12-30|
|  6|      106|   Edward Hudson|  male|   1983-05-09|
|  7|      107|    Albert Jones|  male|   1990-09-13|
|  8|      108|Leonard Cavender|  male|   1958-08-08|
|  9|      109|  Everett Vadala|  male|   2005-05-24|
+---+---------+----------------+------+-------------+
only showing top 10 rows



In [ ]:
df_data_1.take(5)

[Row(_c0=0, person_id=100, name='Penelope Lewis', sex='female', date of birth='1990-08-31'),
 Row(_c0=1, person_id=101, name='David Anthony', sex='male', date of birth='1971-10-14'),
 Row(_c0=2, person_id=102, name='Ida Shipp', sex='female', date of birth='1962-05-24'),
 Row(_c0=3, person_id=103, name='Joanna Moore', sex='female', date of birth='2017-03-10'),
 Row(_c0=4, person_id=104, name='Lisandra Ortiz', sex='female', date of birth='2020-08-05')]

You can also add 5000_points.txt dataset by applying same procedure but click insert to code then click insert credentials then write "file" and "bucket" values inside "path_5000 = cos.url('file_name', 'bucket_name')" expression and comment out path_5000.

In [ ]:
path_5000 = "drive/My Drive/Colab Notebooks/pySpark/Getting Started with PySpark/5000_points.txt"
#df_people = spark.sparkContext.textFile(path_5000)

## Create PySpark DataFrame from external file

We can create PySpark DataFrame by using SparkSession's read.csv method. To do this, we should give path of csv file as an argument to the method. Show action prints first 20 rows of DataFrame. Count action prints number of rows in DataFrame. Columns attribute prints the list of columns in DataFrame. PrintSchema action prints the types of columns in the Dataframe and it gives information about whether there is null values in columns or not.

In [ ]:
df_people = spark.read.csv(path_people, header=True, inferSchema=True)

In [ ]:
df_people.show(10)

+---+---------+----------------+------+-------------+
|_c0|person_id|            name|   sex|date of birth|
+---+---------+----------------+------+-------------+
|  0|      100|  Penelope Lewis|female|   1990-08-31|
|  1|      101|   David Anthony|  male|   1971-10-14|
|  2|      102|       Ida Shipp|female|   1962-05-24|
|  3|      103|    Joanna Moore|female|   2017-03-10|
|  4|      104|  Lisandra Ortiz|female|   2020-08-05|
|  5|      105|   David Simmons|  male|   1999-12-30|
|  6|      106|   Edward Hudson|  male|   1983-05-09|
|  7|      107|    Albert Jones|  male|   1990-09-13|
|  8|      108|Leonard Cavender|  male|   1958-08-08|
|  9|      109|  Everett Vadala|  male|   2005-05-24|
+---+---------+----------------+------+-------------+
only showing top 10 rows



In [ ]:
df_people.count()

100000

In [ ]:
print(len(df_people.columns))

5


In [ ]:
df_people.columns

['_c0', 'person_id', 'name', 'sex', 'date of birth']

In [ ]:
df_people.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- person_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- date of birth: string (nullable = true)



In [ ]:
df_people.drop("name","date of birth").printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- person_id: integer (nullable = true)
 |-- sex: string (nullable = true)



We can use select method to select some columns of DataFrame. If we give argument to show method, it prints out rows as number of argument. In the following example it prints out 10 rows. dropDuplicates method removes the duplicate rows of a DataFrame. We can use count action to see how many rows are dropped.

In [ ]:
df_people_sub = df_people.select('name', "sex", 'date of birth')

In [ ]:
df_people_sub.show(10)

+----------------+------+-------------+
|            name|   sex|date of birth|
+----------------+------+-------------+
|  Penelope Lewis|female|   1990-08-31|
|   David Anthony|  male|   1971-10-14|
|       Ida Shipp|female|   1962-05-24|
|    Joanna Moore|female|   2017-03-10|
|  Lisandra Ortiz|female|   2020-08-05|
|   David Simmons|  male|   1999-12-30|
|   Edward Hudson|  male|   1983-05-09|
|    Albert Jones|  male|   1990-09-13|
|Leonard Cavender|  male|   1958-08-08|
|  Everett Vadala|  male|   2005-05-24|
+----------------+------+-------------+
only showing top 10 rows



In [ ]:
df_people_sub_nodup = df_people_sub.dropDuplicates()

In [ ]:
print("Before removing duplicates:", df_people_sub.count())
print("After removing dublicates:", df_people_sub_nodup.count())

Before removing duplicates: 100000
After removing dublicates: 99998


We can filter out the rows based on a condition by using filter transformation as in the following example.

In [ ]:
df_people_female = df_people.filter(df_people.sex == "female")
df_people_male = df_people.filter(df_people.sex == "male")

In [ ]:
df_people_female.count()

49014

In [ ]:
df_people_female.show(5)

+---+---------+-----------------+------+-------------+
|_c0|person_id|             name|   sex|date of birth|
+---+---------+-----------------+------+-------------+
|  0|      100|   Penelope Lewis|female|   1990-08-31|
|  2|      102|        Ida Shipp|female|   1962-05-24|
|  3|      103|     Joanna Moore|female|   2017-03-10|
|  4|      104|   Lisandra Ortiz|female|   2020-08-05|
| 11|      111|Annabelle Rosseau|female|   1989-07-13|
+---+---------+-----------------+------+-------------+
only showing top 5 rows



We can group columns based on their values by using groupby transformation as in the following example.

In [ ]:
df_people_sex = df_people.groupby("sex")
df_people_sex.count().show()

+------+-----+
|   sex|count|
+------+-----+
|  null| 1920|
|female|49014|
|  male|49066|
+------+-----+



We can sorts the DataFrame based on one or more columns by using orderBy transformation.

In [ ]:
df_people.orderBy("date of birth").show(3)

+-----+---------+---------------+------+-------------+
|  _c0|person_id|           name|   sex|date of birth|
+-----+---------+---------------+------+-------------+
|57359|    57459|   Sharon Perez|female|   1899-08-28|
|62233|    62333|Martina Morison|female|   1901-04-21|
|96318|    96418|   Lisa Garrett|female|   1901-05-09|
+-----+---------+---------------+------+-------------+
only showing top 3 rows



We can rename a column in DataFrame by using withColumnRenamed transformation.

In [ ]:
df_people = df_people.withColumnRenamed("date of birth", "birth")
df_people.show(3)

+---+---------+--------------+------+----------+
|_c0|person_id|          name|   sex|     birth|
+---+---------+--------------+------+----------+
|  0|      100|Penelope Lewis|female|1990-08-31|
|  1|      101| David Anthony|  male|1971-10-14|
|  2|      102|     Ida Shipp|female|1962-05-24|
+---+---------+--------------+------+----------+
only showing top 3 rows



## Using SQL queries with DataFrames by using Spark SQL module

We can also use SQL queries to achieve the same things with DataFrames. Firstly, we should create temporary table by using createOrReplaceTempView method. We should give the name of temporary table as an argument to the method. Then, we can give any query we want to execute to SparkSession's sql method as an argument. Look at the following example.

In [11]:
path_people="/content/drive/MyDrive/Colab Notebooks/Certificado Data Science y AI (DSA) Live/people.csv"
df_people = spark.read.csv(path_people, header=True, inferSchema=True)
df_people = df_people.withColumnRenamed("date of birth", "birth")
df_people.createOrReplaceTempView("people")

In [12]:
df_people.show(10)

+---+---------+----------------+------+-------------------+
|_c0|person_id|            name|   sex|              birth|
+---+---------+----------------+------+-------------------+
|  0|      100|  Penelope Lewis|female|1990-08-31 00:00:00|
|  1|      101|   David Anthony|  male|1971-10-14 00:00:00|
|  2|      102|       Ida Shipp|female|1962-05-24 00:00:00|
|  3|      103|    Joanna Moore|female|2017-03-10 00:00:00|
|  4|      104|  Lisandra Ortiz|female|2020-08-05 00:00:00|
|  5|      105|   David Simmons|  male|1999-12-30 00:00:00|
|  6|      106|   Edward Hudson|  male|1983-05-09 00:00:00|
|  7|      107|    Albert Jones|  male|1990-09-13 00:00:00|
|  8|      108|Leonard Cavender|  male|1958-08-08 00:00:00|
|  9|      109|  Everett Vadala|  male|2005-05-24 00:00:00|
+---+---------+----------------+------+-------------------+
only showing top 10 rows



In [ ]:
#Not working for some reason

query='ALTER TABLE people CHANGE birth date_of_birth'
#query='ALTER TABLE people RENAME COLUMN COLUMN birth TO date_of_birth'
df_people = spark.sql(query).show(10)

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
query='DESCRIBE people'
spark.sql(query).show(10)

+---------+---------+-------+
| col_name|data_type|comment|
+---------+---------+-------+
|      _c0|      int|   null|
|person_id|      int|   null|
|     name|   string|   null|
|      sex|   string|   null|
|    birth|   string|   null|
+---------+---------+-------+



In [15]:
#Use inverted quotes for column names with spaces and triple quote (""") for query string
query="""SELECT name, `birth` FROM people WHERE sex=="male" ORDER BY `birth`"""
df_people_names = spark.sql(query)
df_people_names.show(10)

+---------------+-------------------+
|           name|              birth|
+---------------+-------------------+
|   Tyler Walton|1903-07-14 00:00:00|
|  Daniel Naiman|1903-11-07 00:00:00|
|   John Merritt|1906-11-04 00:00:00|
|   Roger Watkin|1907-12-08 00:00:00|
|     Tim Makris|1909-07-11 00:00:00|
|    Jeremy Jost|1910-04-14 00:00:00|
|  Fredrick Nass|1911-01-12 00:00:00|
|     Shaun King|1911-03-27 00:00:00|
|Mitchell Martin|1911-07-06 00:00:00|
|   Daniel Rutan|1911-08-09 00:00:00|
+---------------+-------------------+
only showing top 10 rows



In [18]:
query='SELECT name, `birth` FROM people WHERE `birth` BETWEEN "1903-01-01" AND "1911-12-31" ORDER BY `birth`'
df_people_1903_1906 = spark.sql(query)
df_people_1903_1906.show(10)

+---------------+-------------------+
|           name|              birth|
+---------------+-------------------+
|   Tyler Walton|1903-07-14 00:00:00|
|  Daniel Naiman|1903-11-07 00:00:00|
| Christy Dawson|1904-01-11 00:00:00|
|   John Merritt|1906-11-04 00:00:00|
|   Roger Watkin|1907-12-08 00:00:00|
|   Marie Givens|1908-02-15 00:00:00|
|Maribel Donahoe|1908-11-27 00:00:00|
|    Paula Evans|1909-02-10 00:00:00|
|     Tim Makris|1909-07-11 00:00:00|
|   Joyce Jacoby|1909-09-13 00:00:00|
+---------------+-------------------+
only showing top 10 rows



In [19]:
query='SELECT sex,COUNT(sex) FROM people WHERE birth BETWEEN "1903-01-01" AND "1911-12-31" GROUP BY sex'
df_people_1903_1906_sex = spark.sql(query)
df_people_1903_1906_sex.show()

+------+----------+
|   sex|count(sex)|
+------+----------+
|female|         7|
|  male|        10|
+------+----------+



## Create RDD from external file

The second and mostly used way to create RDD is from external dataset. To do this, we can use SparkContext's textFile method. In the following example, we use 5000_points.txt dataset. To do this, we use path to dataset as an argument to textFile method.

In [ ]:
# load the 5000_points dataset into a rdd named clusterRDD
clusterRDD = sc.textFile(path_5000)
#NOT clusterRDD.show(10)
clusterRDD.take(10)

Default number of partition of rdd is two. We can see that using getNumPartitions method. If we want to increase number of partition, we can give minPartitions argument to textFile method.

In [ ]:
clusterRDD.getNumPartitions()

In [ ]:
clusterRDD = sc.textFile(path_5000,  minPartitions = 5)

In [ ]:
clusterRDD.getNumPartitions()

In [ ]:
clusterRDD.take(5)

We can transform this rdd by splitting the lines based on the tab.

In [ ]:
rdd_split = clusterRDD.map(lambda x: x.split("\t"))

In [ ]:
rdd_split.take(5)

We can also further transform the splitted RDD to create a list of integers for the two columns.

In [ ]:
rdd_split_int = rdd_split.map(lambda x: [int(x[0]), int(x[1])])

In [ ]:
rdd_split_int.take(5)

## Machine Learning with PySpark MLlib

PySpark MLlib is the Apache Spark's scalable machine learning library in Python consisting of common learning algorithms and utilities. We use Kmeans algorithm of MLlib library to cluster data in 5000_points.txt dataset. First, we should define error method to calculate distance from every point to center of its clusters which the points belong to.

In [ ]:
def error(point):
    center = model.centers[model.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

We train the model with 4 different number of clusters from 13 to 16 and then calculate the error for all of them. As you see in the output, 16 clusters give minimum error. We retrain the model with the number of cluster with the smallest error. We then use clusterCenters attribute to see the center of all clusters.

In [ ]:

# Train the model with clusters from 13 to 16 and compute SSE
temp = 0
cluster = 0
for clst in range(13, 17):
    model = KMeans.train(rdd_split_int, clst, seed=1)
    SSE = rdd_split_int.map(lambda point: error(point)).reduce(lambda x, y: x + y) 
    while temp > SSE or temp == 0:
        temp = SSE
        cluster = clst
    print("The cluster", clst, "has Sum of Squared Error", SSE)


print("\ncluster", cluster)
print("SSE:", temp)


# Train the model again with the best k 
model = KMeans.train(rdd_split_int, k=cluster, seed=1)

# Get cluster centers
cluster_centers = model.clusterCenters

We can again use SparkSession's createDataFrame method to create DataFrame from rdd. We must convert PySpark DataFrame to Pandas DataFrame in order to visualise data. To do this, we can use toPandas method. We create another Pandas DataFrame from cluster centers list. Then, using matplotlib's scatter method, we can make plot for clusters and their centers.

In [ ]:
rdd_split_int_df = spark.createDataFrame(rdd_split_int, schema=["col1", "col2"])

rdd_split_int_df_pandas = rdd_split_int_df.toPandas()

cluster_centers_pandas = pd.DataFrame(cluster_centers, columns=["col1", "col2"])

plt.scatter(rdd_split_int_df_pandas["col1"], rdd_split_int_df_pandas["col2"])
plt.scatter(cluster_centers_pandas["col1"], cluster_centers_pandas["col2"], color="red", marker="x")
plt.show()

## Conclusion

This tutorial discusses Big Data via PySpark, a Python package for spark programming. In this tutorial, I explained SparkContext, using map and filter methods with Lambda functions in Python, creating rdd from object and external file, transformations and actions on rdd and pair rdd, SparkSession, creating PySpark Dataframe from rdd and external file, using sql queries with Dataframes by using Spark SQL module and finally, machine learning with PySpark MLlib library.

In [ ]:
#Converting between Pandas DF, Spark DF and RDD
# load the 5000_points dataset into a rdd named clusterRDD
spRDD = sc.textFile(path_5000)
#NOT clusterRDD.show(10)
spRDD.take(10)
# RDD to Spark DataFrame
spRDD = spRDD.map(lambda x: x.split("\t"))
spRDD = spRDD.map(lambda x: [int(x[0]), int(x[1])])
spRDD.take(10)

#Convert to Spark DF
sparkDF = spRDD.toDF()
sparkDF.show(10)

#Convert to Pandas DF
pdDF = sparkDF.toPandas()
pdDF.head(10)

In [ ]:
#Converting between Pandas DF, Spark DF and RDD
#Converting Pandas DF to Spaek DF
spDF_2 = sqlContext.createDataFrame(pdDF)
spDF_2.show(10)

pdDF.head(10)

#Accesing RDD
spRDD_2=spDF_2.rdd
spRDD_2.take(10)